In [13]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import (VotingClassifier, BaggingClassifier, 
                              RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier)

In [14]:
data = load_breast_cancer(as_frame=True)
X, y = data.data, data.target

print("Dataset Shape:", X.shape)
print("Target Classes:", list(data.target_names))
print("Class Distribution:")
print(y.value_counts())

Dataset Shape: (569, 30)
Target Classes: [np.str_('malignant'), np.str_('benign')]
Class Distribution:
target
1    357
0    212
Name: count, dtype: int64


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
models = {
    'Voting (LR+DT+SVM)': VotingClassifier(estimators=[
        ('lr', LogisticRegression()), 
        ('dt', DecisionTreeClassifier(max_depth=3)), 
        ('svm', SVC())], voting='hard'),
    
    'Bagging (Tree)': BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=50, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=50, random_state=42)
}

In [17]:
print("ENSEMBLE PERFORMANCE")

for name, model in models.items():
    # Train
    model.fit(X_train, y_train)
    
    # Test Accuracy
    acc = accuracy_score(y_test, model.predict(X_test))
    
    # Cross-Validation (5-fold)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)
    
    print(f"\nModel: {name}")
    print(f"  Accuracy: {acc:.4f}")
    print(f"  CV Score: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

ENSEMBLE PERFORMANCE

Model: Voting (LR+DT+SVM)
  Accuracy: 0.9825
  CV Score: 0.9824 (+/- 0.0100)

Model: Bagging (Tree)
  Accuracy: 0.9415
  CV Score: 0.9650 (+/- 0.0339)

Model: Random Forest
  Accuracy: 0.9240
  CV Score: 0.9625 (+/- 0.0353)

Model: AdaBoost
  Accuracy: 0.9591
  CV Score: 0.9675 (+/- 0.0302)

Model: Gradient Boosting
  Accuracy: 0.9474
  CV Score: 0.9524 (+/- 0.0382)
